# End-to-end backend run

## Explanation

This notebook contains an end-to-end run of the code that runs in the backend of our Streamlit app. We developed this backend using Databricks, and later fleshed it out as a fully fledged app with a user interface developed locally.

## Step 1. Necessary installs

In [ ]:
dbutils.library.restartPython()

In [ ]:
%pip install langchain openai streamlit pandas

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 136.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 112.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 130.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 122.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 30.8 MB/s eta 0:00:00
     ━━━━

## Step 2. Necessary imports

In [ ]:
import os
os.chdir("/Workspace/Users/pablo.siso@artefact.com/.ide/artefact-hackathon-03-af6736e3/src")

from generate_persona import generate_persona
from generate_marketing_mix import generate_marketing_mix
from mmx_dict_to_dataframe import mmx_dict_to_dataframe
import pandas as pd
import json

## Step 3. Generate marketing mix strategy from example input

In [ ]:
# OpenAI secret key (necessary to make the code work and not included in repo)
openai_api_key = "your-key"

Call MMX Generation function

In [ ]:
# Open sample input for marketing mix strategy
with open("sample_ins_and_outs/sample_mmx_input.json", "r") as f:
    mmix_strategy_input = json.loads(f.read())

# with open("sample_ins_and_outs/sample_mmx_output.json", "r") as f:
#     strategy = json.loads(f.read())

strategy = generate_marketing_mix(product_attributes=mmix_strategy_input)

In [ ]:
strategy

{'You should definitely invest on': [['YouTube',
   "YouTube is a powerful platform to visually showcase the brewing process, unique flavors, and the story behind Satan's Breath. Creating engaging video content such as behind-the-scenes brewery tours, tasting sessions, and interviews with the brewers can help build brand awareness and connect with the target audience."],
  ['SEO',
   "Investing in SEO is crucial to ensure that when beer enthusiasts in the Netherlands search for craft IPAs or unique beer flavors, Satan's Breath appears at the top of search engine results. Optimizing website content with relevant keywords, creating high-quality backlinks, and implementing local SEO strategies can drive organic traffic to the website."],
  ['Email',
   "Email marketing can be highly effective in nurturing relationships with customers and promoting new IPA releases, special events at taprooms, and exclusive offers. Building an email list through website sign-ups and offering personalized r

Transform JSON to DF Table

In [ ]:
mmx_run_df = mmx_dict_to_dataframe(strategy)

Write DF to Databricks table

In [ ]:
spark.createDataFrame(mmx_run_df).write.mode("append").saveAsTable("default.mmx_runs")

## Step 4. Generate marketing persona from example input

In [ ]:
with open("sample_ins_and_outs/sample_persona_input.json", "r") as f:
    persona_input = json.loads(f.read())

with open("sample_ins_and_outs/sample_persona_output.json", "r") as f:
    persona_output = json.loads(f.read())

# persona_output = generate_persona(mmix_strategy_input, persona_attributes=persona_input)

In [ ]:
persona_output

{'Name': 'Marco, the Italian Beer Connoisseur',
 'StrugglesAndPains': 'Marco craves for distinctive ale flavors and quality that are difficult to find in regular supermarket brands or local bars. He has a strong passion for gastronomy, but is often frustrated by the lack of creativity and uniqueness in the beer industry. Traveling extensively, he struggles to find a brand that provides consistent quality and unique flavors wherever he goes.',
 'OtherProducts': 'Marco often explores a variety of artisanal food and beverage products, ranging from gourmet cheeses and cured meats to premier wines and unique coffee blends. He also enjoys shopping for high-end travel accessories, showing his taste for quality and style.',
 'Goals': 'Marco aspires to discover and enjoy beers that offer non-conventional tastes, tailored towards beer enthusiasts like him. He wants to associate with a brand that values quality and craftsmanship in their brewing techniques. His keen interest also lies in sharing 

In [ ]:
persona_run_df = pd.DataFrame(persona_output, index=[0])

In [ ]:
spark.createDataFrame(persona_run_df).write.mode("append").saveAsTable("default.persona_runs")